## Plot maps

Scripts for plotting maps in Figure 2.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# Read in social media results
# Note! This source layer is not available in the demo
data = pd.read_csv('results_by_country_w_visitor_stats.csv')

In [3]:
# World regions
fp = r"./../world/Global_regions.gpkg"
world = gpd.read_file(fp)

In [4]:
# Check original coordinate reference system
world.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
world.head()

,Country,Cntry_code,FIPS,RegioName,RegCode,SubregCode,OriginUnit,SubReg_2,FID_C_SREG,F_C_R_SR_S,geometry
0,None,0,None,Central America,19,13,Clipperton Island,13,0_0_13,0_0_19_13_13,"MULTIPOLYGON (((-109.22542 10.32042, -109.2254..."
1,None,0,None,Central Asia,142,143,Caspia Sea,143,1_0_143,1_0_142_143_143,"MULTIPOLYGON (((52.86971 40.78276, 52.86163 40..."
2,None,0,None,South-Eastern Asia,142,35,Spratly Islands,35,2_0_35,2_0_142_35_35,"MULTIPOLYGON (((114.36584 10.18056, 114.36611 ..."
3,Afghanistan,4,AF,Southern Asia,142,34,Afghanistan,340,3_4_340,3_4_142_34_340,"MULTIPOLYGON (((71.00423 38.47541, 71.00586 38..."
4,Albania,8,AL,Southern Europe,150,39,Albania,3900,4_8_3900,4_8_150_39_3900,"MULTIPOLYGON (((19.99181 39.77597, 19.99181 39..."


In [6]:
# Set fig size
#plt.rcParams['figure.figsize'] = [24, 12]

In [7]:
from pyproj import CRS

# Define projection Eckert IV from https://spatialreference.org/ref/esri/54012/
# eckert_IV = CRS.from_proj4("+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
robinson = CRS.from_proj4("+proj=robin +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")
# Re-project and plot
world = world.to_crs(robinson)

In [8]:
# Combine statsitics to the polygons
world = world.merge(data, left_on="FIPS", right_on="FIPS")

In [9]:
#world.set_index("FIPS", inplace=True)

In [10]:
world["RegioName"].str.contains("Antarctica")

0      False
1      False
2      False
3      False
4      False
       ...  
717    False
718    False
719    False
720    False
721    False
Name: RegioName, Length: 722, dtype: bool

In [11]:
# Remove antarctica
world = world.loc[(world["RegioName"].str.contains("Antartica"))==False]

In [12]:
world.head()

,Country,Cntry_code,FIPS,RegioName,RegCode,SubregCode,OriginUnit,SubReg_2,FID_C_SREG,F_C_R_SR_S,...,H_dbscan_500km,H_EllipseCentroids,H_maxdays,H_maxmonths,H_maxposts,H_maxtimedelta,H_maxweeks,H_MeanCenters,H_MedianCenters,YEAR2014
0,Afghanistan,4,AF,Southern Asia,142,34,Afghanistan,340,3_4_340,3_4_142_34_340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1,Albania,8,AL,Southern Europe,150,39,Albania,3900,4_8_3900,4_8_150_39_3900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Algeria,12,AG,Northern Africa,2,15,Algeria,15,5_12_15,5_12_2_15_15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
3,American Samoa,16,AQ,Polynesia,9,61,American Samoa,61,6_16_61,6_16_9_61_61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,269
5,Andorra,20,AN,Southern Europe,150,39,Andorra,39,7_20_39,7_20_150_39_39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [13]:
# Disable interactive plotting (it takes a moment to plot the maps..)
plt.ioff()

In [19]:
# PLOT MAXTIMEDELTA
fig, ax = plt.subplots(figsize=(24,12))

world.plot(ax=ax,color="white", facecolor="none", edgecolor="black", linewidth=0.03)

world.plot(ax=ax, column="H_maxtimedelta", cmap="Blues", scheme="quantiles", edgecolor="black", linewidth=0.03, legend=True, 
           missing_kwds={"color" : "white", "edgecolor": "red", "hatch": "////", "label": "No data"},
          legend_kwds={'loc': 'lower left', 'fontsize': '20', 'title': "Origins per country", 'title_fontsize': '22'})

plt.axis('off')
plt.title("Max timedelta (hierarchical)", fontsize=20)

# Crop the figure
ax.set_xlim(-17000000, 17000000)
ax.set_ylim(-8880000, 8880000)

plt.savefig("fig/World_map_H_maxtimedeltaFINAL1.png", bbox_inches='tight')
plt.savefig("fig/World_map_H_maxtimedeltaFINAL1.svg", bbox_inches='tight')

plt.close('all')


In [15]:
# PLOT MAXDAYS
fig, ax = plt.subplots(figsize=(24,12))

world.plot(ax=ax,color="white", facecolor="none", edgecolor="black", linewidth=0.03)

world.plot(ax=ax, column="H_maxdays", cmap="Blues", scheme="quantiles", edgecolor="black", linewidth=0.03, legend=True, 
           missing_kwds={"color" : "white", "edgecolor": "red", "hatch": "////", "label": "No data"},
          legend_kwds={'loc': 'lower left', 'fontsize': '20', 'title': "Visitors per country", 'title_fontsize': '22'})

plt.axis('off')
plt.title("Max days (hierarchical)", fontsize=20)

# Crop the figure
ax.set_xlim(-17000000, 17000000)
ax.set_ylim(-8880000, 8880000)

plt.savefig("fig/World_map_H_maxdaysFINAL1.png", bbox_inches='tight')
plt.savefig("fig/World_map_H_maxdaysFINAL1.svg", bbox_inches='tight')

plt.close('all')



In [16]:
#MAXMONTHS
fig, ax = plt.subplots(figsize=(24,12))

world.plot(ax=ax, color="white", facecolor="none", edgecolor="black", linewidth=0.03)

world.plot(ax=ax, column="H_maxmonths", cmap="Blues", scheme="quantiles", edgecolor="black", linewidth=0.03, legend=True, 
           missing_kwds={"color": "white", "edgecolor": "red", "hatch": "////", "label": "No data"},
          legend_kwds={'loc': 'lower left', 'fontsize': '20', 'title': "Visitors per country", 'title_fontsize': '22'})

plt.axis('off')
plt.title("Max months (hierarchical)", fontsize=20)

# Crop the figure
ax.set_xlim(-17000000, 17000000)
ax.set_ylim(-8880000, 8880000)

plt.savefig("fig/World_map_H_maxmonthsFINAL1.png", bbox_inches='tight')
plt.savefig("fig/World_map_H_maxmonthsFINAL1.svg", bbox_inches='tight')

plt.close('all')



In [17]:
# PLOT MEDIAN CENTERS
fig, ax = plt.subplots(figsize=(24,12))

world.plot(ax=ax,color="white", facecolor="none", edgecolor="black", linewidth=0.03)

world.plot(ax=ax, column="H_MedianCenters", cmap="Blues", scheme="quantiles", edgecolor="black", linewidth=0.03, legend=True, 
           missing_kwds={"color" : "white", "edgecolor": "red", "hatch": "////", "label": "No data"},
          legend_kwds={'loc': 'lower left', 'fontsize': '20', 'title': "Visitors per country", 'title_fontsize': '22'})

plt.axis('off')
plt.title("Median centers (hierarchical)", fontsize=20)

# Crop the figure
ax.set_xlim(-17000000, 17000000)
ax.set_ylim(-8880000, 8880000)

plt.savefig("fig/World_map_H_MedianCentersFINAL1.png", bbox_inches='tight')
plt.savefig("fig/World_map_H_MedianCentersFINAL1.svg", bbox_inches='tight')

plt.close('all')


In [18]:
fig, ax = plt.subplots(figsize=(24,12))

world.plot(ax=ax,color="white", facecolor="none", edgecolor="black", linewidth=0.03)
world.plot(ax=ax, column="YEAR2014", cmap="Blues", scheme="quantiles", edgecolor="black",linewidth=0.03, legend=True,
          legend_kwds={'loc': 'lower left', 'fontsize': '20', 'title': "Visitors per country", 'title_fontsize': '22'})


plt.axis('off')
plt.title("Visitor statistics", fontsize=20)

# Crop the figure
ax.set_xlim(-17000000, 17000000)
ax.set_ylim(-8880000, 8880000)

plt.savefig("fig/World_map_2014_visitors_statisticsFINAL1.png", bbox_inches='tight')
plt.savefig("fig/World_map_2014_visitors_statisticsFINAL1.svg", bbox_inches='tight')

plt.close('all')
